In [26]:
from enum import Enum
from dataclasses import dataclass
import random
from typing import Deque

class Cell(Enum):
    EMPTY = ' '
    BLOCKED = 'X'
    START = 'S'
    GOAL = 'G'
    PATH = '*'
    
@dataclass
class MazeLocation:
    row: int
    column: int
    
class Maze:
    def __init__(self, rows=10, columns=10, sparsity=0.2, start=MazeLocation(0,0), goal=MazeLocation(9,9)):
        self.rows = rows
        self.columns = columns
        self.sparsity = sparsity
        self.start = start
        self.goal = goal
        self.grid = [[Cell.EMPTY for _ in range(self.columns)] for _ in range(self.rows)]
        self.random_fill()
        self.grid[start.row][start.column] = Cell.START
        self.grid[goal.row][goal.column] = Cell.GOAL
    def random_fill(self):
        for row in range(self.rows):
            for col in range(self.columns):
                if random.uniform(0,1) < self.sparsity:
                    self.grid[row][col] = Cell.BLOCKED
    def __str__(self):
        out = ''
        for i in range(self.rows):
            out += ''.join([c.value for c in self.grid[i]]) + '\n'
        return out
    def goal_test(self, ml):
        return ml == self.goal
    def successors(self, ml):
        locations = []
        if ml.row + 1 < self.rows and self.grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.column + 1 < self.columns and self.grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.row - 1 >= 0 and self.grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column - 1 >=0 and self.grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    def mark(self, path, marker):
        for ml in path:
            self.grid[ml.row][ml.column] = marker
        self.grid[self.start.row][self.start.column] = Cell.START
        self.grid[self.goal.row][self.goal.column] = Cell.GOAL
    
    
class Stack:
    def __init__(self):
        self.container = []
    @property
    def is_empty(self):
        return not self.container
    def push(self, item):
        self.container.append(item)
    def pop(self):
        return self.container.pop()
    def __repr__(self):
        return repr(self.container)
    
class Queue:
    def __init__(self):
        self.container = Deque([])
    @property
    def is_empty(self):
        return not self.container
    def push(self, item):
        self.container.append(item)
    def pop(self):
        return self.container.popleft()
    def __repr__(self):
        return repr(self.container)
    
class Node:
    def __init__(self, state, parent, cost=0.0, heuristic=0.0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    
def node2path(node):
    if node is None:
        print('no solution')
        return []
    path = [node.state]
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    return path[::-1]

def dfs(initial, goal_test, successors):
    frontier = Stack()
    frontier.push(Node(initial, None))
    visited = [initial]
    while not frontier.is_empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        for child in successors(current_state):
            if child in visited:
                continue
            visited.append(child)
            frontier.push(Node(child, current_node))
    return None
    
def bfs(initial, goal_test, successors):
    frontier = Queue()
    frontier.push(Node(initial, None))
    visited = [initial]
    while not frontier.is_empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        for child in successors(current_state):
            if child in visited:
                continue
            visited.append(child)
            frontier.push(Node(child, current_node))
    return None
        

In [28]:
n = 30
maze = Maze(rows=n, columns=n, goal=MazeLocation(n-1,n-1))
path = node2path(dfs(maze.start, maze.goal_test, maze.successors))
maze.mark(path, Cell.PATH)
print(maze)
maze.mark(path, Cell.EMPTY)
path = node2path(bfs(maze.start, maze.goal_test, maze.successors))
maze.mark(path, Cell.PATH)
print(maze)
maze.mark(path, Cell.EMPTY)

S***X  X X*********X****X*****
   *****X**   X X *** X***   *
  XX   *** XX      X  X  X****
X ***      **********X X*** X 
 X*X*** XXX*  X X XX*****XX  X
***  X******    X***X       X 
*  ***  XXX *** X* *X*****    
**** ******** *X**X***X  *  X 
X X   X       ***  X  ****XX X
       XX   XXXXX****** X     
     X          X* XX    X  XX
X*******     X X** ***** XX   
X*    X******X X* **X  *XX    
** ****  X X***** *   X*X     
* **X ****X      **  X**  X   
***      *******X* **** X   X 
          X X  *** *X    X    
          X****   **X X    X X
*****    X** X*****   X  XXX  
*  X*******        ****X   X X
***XXX X  X  X X****  **XXXX X
X ****** X*******  X XX**X X  
    X  *XX*           X **X  X
 *******XX*   XX     XX  **** 
 *XX X   X*  XX  XX    XX   *X
X****X   X*     XXX  XX  X **X
    * X****    XX         X*  
X X**X *  X   X       X    *X 
  X*  X*  XX  X        X   ***
   *****XX   X  X X   X   X  G

S   X  X X         X    X     
*       X     X X     X       
**XX   